In [1]:
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
from torch.utils.cpp_extension import load_inline
from torch.profiler import profile, record_function, ProfilerActivity
def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs, with_cuda=True,
                       extra_cuda_cflags=["-O3"] if opt else [], verbose=verbose, name="inline_ext")


In [12]:
cuda_src = open("reduce.cu").read()
cpp_src = """
torch::Tensor simple_sum_reduction(torch::Tensor input, char* method);
"""
funcs = ["simple_sum_reduction",
]
ext = load_cuda(cuda_src, cpp_src, funcs)

/home/aki/miniconda3/envs/imitation/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [13]:
x = torch.ones(1024).cuda()

In [14]:
x

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

In [15]:
o1 = ext.simple_sum_reduction(x, "")

In [16]:
x

tensor([1.0240e+03, 1.0000e+00, 2.0000e+00,  ..., 1.0000e+00, 2.0000e+00,
        1.0000e+00], device='cuda:0')

In [17]:
o1

tensor([1024.], device='cuda:0')

In [28]:
def sum_reduction(x):
    return x.sum()

In [ ]:
device = "cuda"
activities = [ProfilerActivity.CUDA]
sort_by_keyword = device + "_time_total"



In [32]:
x = torch.ones(1024).to(device)

In [ ]:
with profile(activities=activities, record_shapes=True) as prof:
    with record_function("model_inference"):
        ext.simple_sum_reduction(x)

print(prof.key_averages().table(sort_by=sort_by_keyword, row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
void at::native::reduce_kernel<512, 1, at::native::R...         0.00%       0.000us         0.00%       0.000us       0.000us       1.984us       100.00%       1.984us       1.984us             1  
                                       cudaLaunchKernel        99.96%       5.349ms        99.96%       5.349ms       5.349ms       0.000us         0.00%       0.000us       0.000us             1  
         